In [1]:
#Inicializacion de spark session y creacion de nuestra instancia
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sesion3').getOrCreate()

In [2]:
empleats = [(1,"Jaime",-1,"2018","10","M",1300), \
    (2,"Maria",1,"2010","20","F",1400), \
    (3,"Rosa",1,"2010","10","F",1000), \
    (4,"Vanesa",2,None,"10","F",1800), \
    (5,"Leonardo",2,"2010","40","M",1300), \
      (6,"Paula",2,"2013","20","F",1600), \
    (7,"Jose Luis",2,"2014","30","F",1200) \
  ]
departaments = [("Financiero",10),("Marketing",20),("Ventas",30),("Desarrollo",40)]

In [3]:
from pyspark.sql.types import StructField,StructType,IntegerType,StringType,FloatType

est_emp = StructType([StructField('emp_id',IntegerType(),False),
              StructField('nom',StringType(),True),
              StructField('superior_emp_id',IntegerType(),True),
              StructField('data_i',StringType(),True),
              StructField('dept_id',StringType(),True),
              StructField('genere',StringType(),True),
              StructField('sou',IntegerType(),True)
             ])
est_dep = StructType([StructField('dept_nom',StringType(),False),
              StructField('dept_id',IntegerType(),False)
          ])


In [4]:
df_emp = spark.createDataFrame(empleats,schema=est_emp)
df_dep = spark.createDataFrame(departaments,schema=est_dep)

In [5]:
df_dep.show()

+----------+-------+
|  dept_nom|dept_id|
+----------+-------+
|Financiero|     10|
| Marketing|     20|
|    Ventas|     30|
|Desarrollo|     40|
+----------+-------+



In [6]:
df_emp.show()

+------+---------+---------------+------+-------+------+----+
|emp_id|      nom|superior_emp_id|data_i|dept_id|genere| sou|
+------+---------+---------------+------+-------+------+----+
|     1|    Jaime|             -1|  2018|     10|     M|1300|
|     2|    Maria|              1|  2010|     20|     F|1400|
|     3|     Rosa|              1|  2010|     10|     F|1000|
|     4|   Vanesa|              2|  null|     10|     F|1800|
|     5| Leonardo|              2|  2010|     40|     M|1300|
|     6|    Paula|              2|  2013|     20|     F|1600|
|     7|Jose Luis|              2|  2014|     30|     F|1200|
+------+---------+---------------+------+-------+------+----+



In [19]:
# 1. Saber ¿Cuál es el sueldo medio de cada departamento?
df_emp.select("dept_id","sou").groupBy("dept_id").avg().orderBy("dept_id").show()

+-------+------------------+
|dept_id|          avg(sou)|
+-------+------------------+
|     10|1366.6666666666667|
|     20|            1500.0|
|     30|            1200.0|
|     40|            1300.0|
+-------+------------------+



In [23]:
# Eliminar las filas que tengan el campo fecha_inicio a nulo
df_emp.na.drop(subset= "data_i").show()

+------+---------+---------------+------+-------+------+----+
|emp_id|      nom|superior_emp_id|data_i|dept_id|genere| sou|
+------+---------+---------------+------+-------+------+----+
|     1|    Jaime|             -1|  2018|     10|     M|1300|
|     2|    Maria|              1|  2010|     20|     F|1400|
|     3|     Rosa|              1|  2010|     10|     F|1000|
|     5| Leonardo|              2|  2010|     40|     M|1300|
|     6|    Paula|              2|  2013|     20|     F|1600|
|     7|Jose Luis|              2|  2014|     30|     F|1200|
+------+---------+---------------+------+-------+------+----+



In [17]:
# 3. A los empleados que tienen más de 10 años de antigüedad se les 
# va a subir el sueldo al doble calcula con una función UDF el nuevo
# sueldo de los empleados y añádelo a una nueva columna Sueldo_nuevofrom pyspark.sql.functions import udf
def doble(s):
    return 2 * s

doble_udf = udf(lambda x: doble(x),IntegerType())

In [18]:
df_emp.where(df_emp["data_i"] <="2011").withColumn('sou_nou',doble_udf(df_emp['sou'])).show()

+------+--------+---------------+------+-------+------+----+-------+
|emp_id|     nom|superior_emp_id|data_i|dept_id|genere| sou|sou_nou|
+------+--------+---------------+------+-------+------+----+-------+
|     2|   Maria|              1|  2010|     20|     F|1400|   2800|
|     3|    Rosa|              1|  2010|     10|     F|1000|   2000|
|     5|Leonardo|              2|  2010|     40|     M|1300|   2600|
+------+--------+---------------+------+-------+------+----+-------+



In [12]:
# Exercici 4
#Sesnse joins queda molt llarg
df_emp.where(df_emp["superior_emp_id"]==(df_emp.where(df_emp['nom']=='Maria').select('emp_id').collect()[0][0])).orderBy(df_emp['sou'].desc()).limit(1).show()

+------+------+---------------+------+-------+------+----+
|emp_id|   nom|superior_emp_id|data_i|dept_id|genere| sou|
+------+------+---------------+------+-------+------+----+
|     4|Vanesa|              2|  null|     10|     F|1800|
+------+------+---------------+------+-------+------+----+



In [35]:

#Exercici 4 amb join  NO M'HA FUNCIONAT. DIU QUE EM FAN FALTA ALIAS
emp1 = df_emp.alias("emp1")
emp2 = df_emp.alias("emp2")
emp1.join(emp2,emp1.superior_emp_id==emp2.emp_id,'INNER').show()

SyntaxError: invalid syntax (<ipython-input-35-0b2ccb8a2c6b>, line 3)

In [14]:
# Exercici 4 me'n vaig a SQL a veure que tal
df_emp.createTempView("emp")
df_dep.createTempView("dep")

AnalysisException: Temporary view 'emp' already exists

In [17]:
spark.sql("SELECT e1.emp_id, e1.nom, e1.sou FROM emp e1 INNER JOIN emp e2 ON e1.superior_emp_id=e2.emp_id WHERE e2.nom='Maria' ORDER BY e1.sou DESC LIMIT 1").show()

+------+------+----+
|emp_id|   nom| sou|
+------+------+----+
|     4|Vanesa|1800|
+------+------+----+



In [15]:
# 5. Crea una vista que se pueda acceder a ella desde otra sesión que muestre el total que se gasta en sueldos por departamentos
df_emp.join(df_dep,df_emp.dept_id==df_dep.dept_id,'INNER').select('dept_nom','sou').groupBy('dept_nom').sum().createGlobalTempView("Departaments")

In [17]:
spark.sql("Select * from global_temp.Departaments").show()

+----------+--------+
|  dept_nom|sum(sou)|
+----------+--------+
|    Ventas|    1200|
| Marketing|    3000|
|Financiero|    4100|
|Desarrollo|    1300|
+----------+--------+



In [7]:
# 6. Se acaba de contratar una jefa de ventas, llamada Elena Martínez,
# va a cobrar 1700, necesitamos añadirla a nuestra base de datos 
df_emp_nova = spark.createDataFrame([(8,"Elena Martínez",-1,"2021","30","F",1700)],schema=est_emp)

In [8]:
df_emp_nova.show()

+------+--------------+---------------+------+-------+------+----+
|emp_id|           nom|superior_emp_id|data_i|dept_id|genere| sou|
+------+--------------+---------------+------+-------+------+----+
|     8|Elena Martínez|             -1|  2021|     30|     F|1700|
+------+--------------+---------------+------+-------+------+----+



In [11]:
df_emp = df_emp.union(df_emp_nova)

In [12]:
df_emp.show()

+------+--------------+---------------+------+-------+------+----+
|emp_id|           nom|superior_emp_id|data_i|dept_id|genere| sou|
+------+--------------+---------------+------+-------+------+----+
|     1|         Jaime|             -1|  2018|     10|     M|1300|
|     2|         Maria|              1|  2010|     20|     F|1400|
|     3|          Rosa|              1|  2010|     10|     F|1000|
|     4|        Vanesa|              2|  null|     10|     F|1800|
|     5|      Leonardo|              2|  2010|     40|     M|1300|
|     6|         Paula|              2|  2013|     20|     F|1600|
|     7|     Jose Luis|              2|  2014|     30|     F|1200|
|     8|Elena Martínez|             -1|  2021|     30|     F|1700|
+------+--------------+---------------+------+-------+------+----+

